In [1]:
#The 1st iteration of the Soybean detection & color analysis Prototype
#Worked on by: Fubin & Bayler

In [2]:
#Imports
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os

%matplotlib inline

#Defining function for getting an image properly
def get_image(image_path):
    image = cv2.imread(image_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

#Defining funciton for convert from RGB to HEX
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

# load image as grayscale
img = cv2.imread('Cropped.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# threshold input image using otsu thresholding as mask and refine with morphology
blur = cv2.GaussianBlur(gray,(5,5),0)
ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU) 
kernel = np.ones((9,9), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# put thresh into 
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2RGBA)
result[:, :, 3] = mask

#Beginning of color analysis section

#Resizing & reshaping the mask
modified_image = cv2.resize(result, (600, 400), interpolation = cv2.INTER_AREA)
modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)

#Adjustments for available colors for analysis
clf = KMeans(n_clusters = 10) #number of colors here
labels = clf.fit_predict(modified_image)
counts = Counter(labels)

#Retrieve ordered colors by iterating through the keys
center_colors = clf.cluster_centers_
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]

#Displaying the pie chart
if (True) :
    plt.figure(figsize = (8, 6))
    plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
rgb_colors

ValueError: cannot reshape array of size 960000 into shape (240000,3)